In [1]:
%%cmd
python --version

Microsoft Windows [Version 10.0.22621.1992]
(c) Microsoft Corporation. All rights reserved.

C:\Users\Priya\face recognition>python --version
Python 3.9.13

C:\Users\Priya\face recognition>

In [2]:
! pip install opencv-python

In [16]:
pip install face-recognition

Note: you may need to restart the kernel to use updated packages.


In [17]:
%%cmd
pip install cmake

Microsoft Windows [Version 10.0.22621.1992]
(c) Microsoft Corporation. All rights reserved.

C:\Users\Priya\face recognition>pip install cmake

C:\Users\Priya\face recognition>

In [19]:
%%cmd
pip install "C:\Users\Priya\Downloads\dlib-19.23.0-cp39-cp39-win_amd64.whl"

Microsoft Windows [Version 10.0.22621.1992]
(c) Microsoft Corporation. All rights reserved.

C:\Users\Priya\face recognition>pip install "C:\Users\Priya\Downloads\dlib-19.23.0-cp39-cp39-win_amd64.whl"
Processing c:\users\priya\downloads\dlib-19.23.0-cp39-cp39-win_amd64.whl
dlib is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.

C:\Users\Priya\face recognition>

In [20]:
%%cmd
pip install face-recognition

Microsoft Windows [Version 10.0.22621.1992]
(c) Microsoft Corporation. All rights reserved.

C:\Users\Priya\face recognition>pip install face-recognition

C:\Users\Priya\face recognition>

In [41]:
!pip install cvzone

In [1]:
import face_recognition

In [2]:
import os

In [3]:
import numpy as np

In [4]:
import cv2

In [5]:
import pickle

In [6]:
import cvzone

In [7]:
import firebase_admin
from firebase_admin import credentials, firestore

In [8]:
from firebase_admin import storage

In [9]:
from firebase_admin import db

In [10]:
from datetime import datetime

In [11]:
try:
    app = firebase_admin.get_app()
except:
    cred = credentials.Certificate("serviceAccountKey.json")
    firebase_admin.initialize_app(cred, {"databaseURL": "https://faceattendancesystem-b673f-default-rtdb.firebaseio.com/",
                                        "storageBucket": "faceattendancesystem-b673f.appspot.com"})

bucket = storage.bucket()

cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

imgBackground = cv2.imread("C:\\Users\\Priya\\face recognition\\Resources\\background.png")

# IMPORTING THE MODE IMAGES INTO A LIST
folderModePath = "C:\\Users\\Priya\\face recognition\\Resources\\Modes"
modePathList = os.listdir(folderModePath)
imgModeList = []

for path in modePathList:
    imgModeList.append(cv2.imread(os.path.join(folderModePath,path)))

# LOAD THE ENCODING FILE
print("loading encode file")
file = open('encodeFile.p','rb')
encodeListWithID = pickle.load(file)
file.close()
encodeListKnown, studentIds = encodeListWithID
# print(studentId)
print("encode file loaded")

    
modeType = 0   
counter = 0
id = -1
imgStudent = []
students = "https://faceattendancesystem-b673f-default-rtdb.firebaseio.com/"

while True:
    success, img = cap.read()
    
    # none is a parameter of dst. if none is provided a new image will be created for the ouput
    # 0.25 are the scaling factors, which means the image will be reduced to one fourth of size 
    imgS = cv2.resize(img,(0,0),None,0.25,0.25)
    imgs = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    
    # DETECT THE FACE FROM THE IMAGE
    faceCurFrame = face_recognition.face_locations(imgS)
    
    # COMPUTE THE FACE ENCODINGS (FACIAL FEATURES) FROM THE FACE
    encodeCurFrame = face_recognition.face_encodings(imgS,faceCurFrame)
    
    imgBackground[162: 162 + 480, 55: 55 + 640] = img
    imgBackground[44: 44 + 633, 808: 808 + 414] = imgModeList[modeType]
    
    if faceCurFrame:
        for encodeFace, faceLoc in zip(encodeCurFrame, faceCurFrame):
            matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
            faceDist = face_recognition.face_distance(encodeListKnown, encodeFace)
             #print("matches", matches)
             #print("face distance", faceDist)

            matchIdx = np.argmin(faceDist)
             #print("match index", matchIdx)

            if matches[matchIdx]:
                 # print("known face detected")
                 # print(studentIds[matchIdx]) 
                y1, x2, y2, x1 = faceLoc
                y1, x2, y2, x1 = y1*4, x2*4, y2*4, x1*4
                
                #(x, y, width, height)
                bbox = 55 + x1, 162 + y1, x2 - x1, y2 - y1
                imgBackground = cvzone.cornerRect(imgBackground,bbox,rt=0)
                id = studentIds[matchIdx]

                if counter == 0:
                    counter = 1
                    modeType = 1

        if counter != 0:
            if counter == 1:
                # GET THE DATA
                studentInfo = db.reference(f'students/{id}').get()
                print(studentInfo)

                # UPDATE THE DATA OF ATTENDANCE
                datetimeObject = datetime.strptime(studentInfo['last_attendance_time'],"%Y-%m-%d %H:%M:%S")
                secondsElapsed = (datetime.now() - datetimeObject).total_seconds()

                if secondsElapsed > 30:
                    ref = db.reference(f'students/{id}')
                    studentInfo['total_attendance'] += 1
                    ref.child('total_attendance').set(studentInfo['total_attendance'])
                    ref.child('last_attendance_time').set(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

                else:
                    modeType = 3
                    counter = 0
                    imgBackground[44: 44 + 633, 808: 808 + 414] = imgModeList[modeType]

            if modeType != 3:
                if 10 < counter < 20:
                    modeType = 2
                    imgBackground[44: 44 + 633, 808: 808 + 414] = imgModeList[modeType]

                if counter <= 10:
                    cv2.putText(imgBackground,str(studentInfo['total_attendance']),(861,125),cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 1)
                    cv2.putText(imgBackground,str(studentInfo['name']),(808,445),cv2.FONT_HERSHEY_COMPLEX, 1, (50, 50, 50), 1)
                    cv2.putText(imgBackground,str(studentInfo['major']),(1006,550),cv2.FONT_HERSHEY_COMPLEX, 0.4, (255, 255, 255), 1)
                    cv2.putText(imgBackground,str(id),(1006,493),cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1)
                    cv2.putText(imgBackground, str(studentInfo['year']), (1025, 625), cv2.FONT_HERSHEY_COMPLEX, 0.6, (100, 100, 100), 1)
                    cv2.putText(imgBackground, str(studentInfo['starting_year']), (1125, 625),cv2.FONT_HERSHEY_COMPLEX, 0.6, (100, 100, 100), 1)


                counter += 1

                if counter >= 20:
                    counter = 0
                    modeType = 0
                    studentInfo = []
                    imgBackground[44: 44 + 633, 808: 808 + 414] = imgModeList[modeType]

    else:
        modeType = 0
        counter = 0
        
    
    cv2.imshow("face attendance",imgBackground)
    cv2.waitKey(1)
    
    if cv2.waitKey(1) == 27:
        break

cap.release()  
cv2.destroyAllWindows()  

loading encode file
encode file loaded
{'last_attendance_time': '2022-11-11 00:57:33', 'major': 'computer science', 'name': 'Priya Gulati', 'starting_year': 2022, 'total_attendance': 9, 'year': 2}
{'last_attendance_time': '2023-08-28 11:33:07', 'major': 'computer science', 'name': 'Priya Gulati', 'starting_year': 2022, 'total_attendance': 10, 'year': 2}
{'last_attendance_time': '2023-08-28 11:33:07', 'major': 'computer science', 'name': 'Priya Gulati', 'starting_year': 2022, 'total_attendance': 10, 'year': 2}
